In [1]:
import gymnasium as gym
import tensorflow as tf
import numpy as np
import random
import collections

### Huber loss

Huber损失是用于稳健回归的损失函数

\begin{equation}
L_\delta = 
\left\{
             \begin{array}{lr}
             \frac{1}{2} (y-f(x))^{2} & |y-f(x)|\le \delta\\
             \delta \cdot (|y-f(x)|-\frac{1}{2}\delta) & otherwise
             \end{array}
\right.
\end{equation}

References: 
> https://en.wikipedia.org/wiki/Huber_loss 

> https://www.tensorflow.org/api_docs/python/tf/losses/huber_loss

#### 探索率

> 探索率就是指系统会采取随机行为的概率.

在强化学习（*reinforcement learning*）中，探索率（*exploration rate*）通常指的是一个参数，用于控制 Agent 在尝试新的行为或者采用已知的最优行为的概率。在许多强化学习算法中，探索率常常使用一个符号为 `ε` 的参数来表示，这个参数被称为 ε-贪心策略（ε-greedy strategy）。

**ε-贪心策略**（*ε-greedy strategy*）是一种用于强化学习（reinforcement learning）的策略，用于控制 Agent 在尝试新的行为或者采用已知的最优行为的概率。在 ε-贪心策略中，**Agent 会以 `1-ε` 的概率采用最优的行为，以 `ε` 的概率选择随机的行为**。这样做的原因是，在探索新的策略的同时，Agent 也可以在大多数情况下采用最优的行为。

#### 折扣率

...

#### 学习率 

...

In [2]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        """
        
        """
        self.memory = collections.deque(maxlen=2000)
        """ discount rate 折扣率 """
        self.gamma = 0.95
        """ exploration rate 探索率 """
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        """ learning rate 学习率 """
        self.learning_rate = 0.001
        
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _huber_loss(self, y_true, y_pred, clip_delta=1.0):
        """
        实现自定义Huber损失函数 loss function

        err = y_true - y_pred
        ret = 0
        if abs(error) <= clip_delta :
            ret = 0.5 * err**2
        else:
            ret = clip_delta * (abs(err) - 0.5 * clip_delta)
        return 
        """
        error = y_true - y_pred
        cond  = tf.keras.backend.abs(error) <= clip_delta

        squared_loss = 0.5 * tf.keras.backend.square(error)
        quadratic_loss = 0.5 * tf.keras.backend.square(clip_delta) + \
                        clip_delta * (tf.keras.backend.abs(error) - clip_delta)

        return tf.keras.backend.mean(tf.where(cond, squared_loss, quadratic_loss))

    def _build_model(self):
        """
        创建神经网络模型
        input_layer:  24个单元,ReLU激活,输入尺寸state_size
        hidden_layer: 24个单元,ReLU激活
        output_layer: ,linear激活
        """
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(units=24, input_dim=self.state_size, activation='relu'))
        model.add(tf.keras.layers.Dense(units=24, activation='relu'))
        model.add(tf.keras.layers.Dense(units=self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss,
                      optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    def update_target_model(self):
        """
        更新模型权重
        copy weights from model to target_model
        """
        self.target_model.set_weights(self.model.get_weights())

    def memorize(self, state, action, reward, next_state, done):
        """
        存储信息 (状态、动作、奖励、下一次状态、游戏是否结束)
        """
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        """
        ......
        """
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        """
        ......
        """
        minibatch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state, done in minibatch:

            target = self.model.predict(state)

            if done:
                target[0][action] = reward
            else:
                # a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
                # target[0][action] = reward + self.gamma * t[np.argmax(a)]

            self.model.fit(state, target, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        """
        加载已有模型
        """
        self.model.load_weights(name)

    def save(self, name):
        """
        保存模型
        """
        self.model.save_weights(name)


...

In [21]:
EPISODES = 5

"""
在CartPole-v1模拟环境中,
    <state>是一个四维的向量.这些信息组合起来,描述了小车杆系统的当前状态.
    {
        x:小车的水平位移,从小车的初始位置(位于屏幕中央)开始测量.
        x_dot:小车的水平速度。
        theta:杆的倾角,从垂直向下的位置开始测量.
        theta_dot:杆的角速度.
    }
    <...>
    {
        ...
    }
    在这个任务中,你的目标是通过控制小车的水平力量，使得杆保持垂直,并尽量延长运行时间.
"""
# create environment
env = gym.make('CartPole-v1')
"""" 获取环境中的可能的状态数量 """
state_size = env.observation_space.shape[0]
""" 获取环境中的可采取的动作数量 """
action_size = env.action_space.n

# 创建代理
agent = DQNAgent(state_size, action_size)
#  加载已有权重模型
# agent.load("./save/cartpole-ddqn.h5")

done = False

batch_size = 32

for i in range(EPISODES):
    
    """
    重置环境,获取当前的状态
    """
    state, _ = env.reset()
    
    # print("\n state:", state, np.shape(state))

    state = np.reshape(state, [1, state_size])

    print("\n state:", state, np.shape(state))

    # 每一个新的环境中执行5000次动作
    for time in range(50):

        """
        render() 函数用于渲染出当前的智能体以及环境的状态
        """
        # env.render()

        """ 根据当前环境做出动作 """
        action = agent.act(state)
        
        """
        在环境中执行当前动作得到反馈信息 
        next_state: 下一个状态信息
        reward:     执行动作后得到的奖励
        done:       一个布尔值,<表示本次游戏是否已经结束>.
                    :当木棒倾斜超过特定角度或者小车的位置超
                    出轨道边界时,游戏结束.
        """
        next_state, reward, done, _, _ = env.step(action)
        
        # reward = reward if not done else -10
        
        """
        解析`next_state`的信息, 并重新计算reward
        x:小车的水平位移,从小车的初始位置(位于屏幕中央)开始测量.
        x_dot:小车的水平速度。
        theta:杆的倾角,从垂直向下的位置开始测量.
        theta_dot:杆的角速度.
        """
        x,x_dot,theta,theta_dot = next_state
        
        """
        ......
        """
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        reward = r1 + r2
        
        next_state = np.reshape(next_state, [1, state_size])
        
        """ ...... """
        agent.memorize(state, action, reward, next_state, done)
        
        """ 状态转换 """
        state = next_state
        
        print('.', end='')
        
        """ 游戏结束："""
        if done:
            # 更新权重模型
            agent.update_target_model()
            # 打印信息
            """
            打印格式 episode: 1/5, score: 36, e: 0.87 
            episode:
            score:游戏得分
            e:
            """
            print("episode: {}/{}, score: {}, e: {:.2}"
                    .format(i, EPISODES, time, agent.epsilon))
            break
        
        """ ...... """
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    # if e % 10 == 0:
    #     agent.save("./save/cartpole-ddqn.h5")


 state: [[-0.04790396  0.00167995  0.03117294  0.04469715]] (1, 4)
..........episode: 0/5, score: 9, e: 1.0

 state: [[-0.00307816  0.02904799  0.04880029  0.01994588]] (1, 4)
.....................................episode: 1/5, score: 36, e: 0.87

 state: [[-0.03839378 -0.03257278  0.01113296 -0.00751979]] (1, 4)
........................episode: 2/5, score: 23, e: 0.69

 state: [[0.00925898 0.01000441 0.03320426 0.01908784]] (1, 4)
...........episode: 3/5, score: 10, e: 0.62

 state: [[-0.0227491  -0.00637077  0.02658374 -0.04080787]] (1, 4)
.......................episode: 4/5, score: 22, e: 0.5
